In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!unzip Flickr8k_Dataset.zip
!rm -rf *.zip
!rm -rf __MACOSX

In [2]:
%cd '/content/drive/MyDrive/ViTCAP'
!pip install -q condacolab
import condacolab
condacolab.install()

/content/drive/MyDrive/ViTCAP
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [ ]:
with open('/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/train.caption.tsv','r') as file:
  test_data = json.load(file)

In [ ]:
with open('/content/test_caption.json','r') as file:
  test_data = json.load(file)

In [ ]:
test1 = test_data[0]
print(test1)

In [ ]:
root_path = "/content/Flicker8k_Dataset/"
extension = '.jpg'

In [ ]:
test_tsv_data = open("test_tsv_data.tsv", "a")

In [ ]:
for data in test_data:
  with open(root_path+data['image_id']+extension, 'rb') as image_file:
      image_data = image_file.read()

  base64_encoded = base64.b64encode(image_data).decode('utf-8')

  test_tsv_data.write(data['image_id'])
  test_tsv_data.write('\t')
  test_tsv_data.write(base64_encoded)
test_tsv_data.close()

In [ ]:
%cd '/content/drive/MyDrive/ViTCAP'

!conda create -n vitcap python=3.8
!conda activate vitcap

In [2]:
!source activate vitcap

In [ ]:
%cd '/content/drive/MyDrive/ViTCAP'

!pip install -r requirements.txt

In [ ]:
!pip install timm
!pip install transformers

!pip install einops

In [4]:
%cd '/content/drive/MyDrive/ViTCAP'

import os
import PIL
import math
import yaml
import torch
import logging
import requests
import numpy as np
import torch.nn as nn
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from IPython.display import Image
import torchvision.transforms as transforms
from src.pipelines.uni_pipeline import get_transform_vit_default
%matplotlib inline


caption_pipeline = './images/caption_pipeline.png'
vit_pipeline = './images/ViTCAP.png'

/content/drive/MyDrive/ViTCAP


In [ ]:
Image(filename=caption_pipeline)


In [ ]:
Image(filename=vit_pipeline)


In [5]:

# Image Captioning model wrapper
class ImageCaptioning(nn.Module):
    def __init__(self,
                 model,
                 test_extra_input=None,
                 tokenizer=None,
                 bert_tokenizer=None,
                 image_encoder=None,
                 cfg=None,
                 ):
        super().__init__()
        self.module = model
        self.iter = 0
        self.tokenizer = tokenizer
        self.bert_tokenizer = bert_tokenizer
        self.test_extra_input = test_extra_input
        self.image_encoder = image_encoder
        self.cfg = cfg
#         self.acc = MultiLabelAccuracy()
#         self.map = mAPMeter()

        if cfg.pert_img_prob is not None and cfg.pert_img_prob > 0:
            # we need an image text matching loss on the pooled output
            # number of relationship is always 1, we use BCE loss
            self.seq_relationship = nn.Linear(model.bert.pooler.dense.weight.shape[0], 1)
            assert self.cfg.mask_type != 'seq2seq', 'matching loss is useless'
        else:
            self.seq_relationship = None

        self.category = cfg.category
        if self.category == 'vinvl':
            self.vocab = self.tokenizer['idx_to_label']
        else:
            self.vocab = self.bert_tokenizer.ids_to_tokens

    def construct_attn_mask(self, data):
        img_feats = data['img_feats']
        input_ids = data['input_ids']
        attention_mask = data['attention_mask']
        batch_size = img_feats.shape[0]

        num_img_feats = img_feats.shape[1]
        num_token = input_ids.shape[-1]
        device = input_ids.device
        top_right = torch.ones((batch_size, num_token, num_img_feats), device=device)
        if self.cfg.mask_type == 'seqbid':
            mask_type = data.pop('mask_type')
            bottom_left = torch.ones((batch_size, num_img_feats, num_token), device=device)
            # if mask_type is 1, it is seq2seq and we need to zero it out
            bottom_left[mask_type] = 0
        elif self.cfg.mask_type in ['seq2seq', 'seq2seq_off']:
            bottom_left = torch.zeros((batch_size, num_img_feats, num_token), device=device)
        else:
            assert self.cfg.mask_type == 'bidirectional'
            bottom_left = torch.ones((batch_size, num_img_feats, num_token), device=device)
            if attention_mask.dim() == 2:
                attention_mask = attention_mask.unsqueeze(dim=1)
                attention_mask = attention_mask.expand(batch_size, num_token, num_token)
        bottom_right = torch.ones((batch_size, num_img_feats, num_img_feats), device=device)
        bottom = torch.cat((bottom_left, bottom_right), dim=2)

        top = torch.cat((attention_mask, top_right), dim=2)
        full_attention_mask = torch.cat((top, bottom), dim=1)
        data['attention_mask'] = full_attention_mask

    def forward(self, data):

        data = dict(data.items())

        # this is required in test, but not in train
        data.pop('key')
        data['gen_tag_ratio'] = 1

        assert self.image_encoder
        assert 'img_feats' not in data
        data['img_feats'] = self.image_encoder(data)

        data.pop('image')
        self.construct_attn_mask(data)

        # Note: this wrapper only contains inference code, refer XX_captioning_uni_pipelineXX.py for full code
        # if self.cfg.use_cbs:
        #     data.update({
        #         'min_constraints_to_satisfy': 2,
        #         'use_cbs': True,
        #     })

        data.update(self.test_extra_input)
        result = self.module(**data)

        return result

In [6]:
# parameter loading
with open(r'/content/drive/MyDrive/ViTCAP/yaml/ViTCAP_Captioning_batch-size_512_encoder_vit_base_patch16_384_lr_1e-4_iter_60_vitbfocal20_bert_tokenizer_tags_ENC-DEC_multiplier_0.1.yaml') as file:
    cfg = yaml.full_load(file)


class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

cfg['param'].update(cfg['all_test_data'])

# update some value
cfg['param'].update({
    'mask_type': 'seq2seq',
    'max_seq_a_length': 40,
    'max_seq_length': 70,
    'add_od_labels': True,
    'od_label_conf ': 0.2,
    'drop_out': 0.1,
    'tie_weights': True,
    'label_smoothing': 0.1,
    'img_layer_norm_eps': 1e-5,
    'max_img_seq_length': 50,
    'max_gen_length': 20,
    'output_isvalid': False,
    'max_masked_tokens': 3,
    'cider_cached_tokens': 'data/coco_caption/gt/coco-train-words.p',
    'num_beams': 1,
    'mask_prob': 0.15,
    'replace_by_mask_prob': 0.8,
    'replace_by_rand_prob': 0.1,
    'temperature': 1,
    'top_k': 0,
    'top_p': 1,
    'gradient_clip': 1.,
    'optimizer_type': 'MAdamW',
    'bias_no_weight_decay': True,
    'ln_no_weight_decay': True,
    'unique_labels_on': False,
    'scheduler_type': 'linear',
    'pad_to_max': True,
    'no_sort_by_conf': False,
    'ignore_project_image': False,
    'real_text_a_in_test': False,
    'pert_img_prob': None,
})

cfg = Struct(**cfg['param'])

In [ ]:
%cd /content/drive/MyDrive/ViTCAP
!python run.py -c ./yaml/ViTCAP_Captioning_batch-size_512_encoder_vit_base_patch16_384_lr_1e-4_iter_60_vitbfocal20_bert_tokenizer_tags_ENC-DEC_multiplier_0.1.yaml


In [ ]:
!pip install future
!pip install matplotlib
!pip install progressbar
!pip install nltk
!pip install boto3
!pip install opencv-python


Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#Load ViTCAP model
%cd '/content/drive/MyDrive/ViTCAP'
from src.layers.bert import BertTokenizer,BertConfig

# property
def tag_vocab():
    assert op.isfile(cfg.tokenizer_file)
    tag_vocab = json.load(open(cfg.tokenizer_file))
    return tag_vocab


# property
def get_tokenizer(cfg):
    tokenizer = BertTokenizer.from_pretrained(
       cfg.text_encoder_type, do_lower_case=True)
    return tokenizer


config = BertConfig.from_pretrained(
    # this class does not support a separate text encoder and thus
    # text_encoder_type here means the joint fusion encoder
    cfg.text_encoder_type,
    num_labels=2,
    finetuning_task='image_captioning',
)


if 'vit' in cfg.text_encoder_type:
    # this is just to make sure we are using the right variables for vit model
    assert cfg.drop_out == 0

config.img_feature_type = 'frcnn'
config.hidden_dropout_prob = cfg.drop_out
config.loss_type = 'classification'
config.tie_weights = False
config.freeze_embedding = False
config.label_smoothing = False
config.drop_worst_ratio = 0
config.drop_worst_after = 0
config.img_feature_dim = cfg.img_feature_dim
config.use_img_layernorm = cfg.use_img_layernorm
config.img_layer_norm_eps = 0
config.net = cfg.image_encoder_type[len('VitEmb_'):]
config.ignore_project_image = cfg.ignore_project_image
config.later_captioning = 7
config.attn_token_sample = 0.1
config.vocab = tag_vocab
config.tokenizer = get_tokenizer(cfg)
config.loss = getattr(cfg, 'loss', 'bce')
config.split_blocks = getattr(cfg, 'split_blocks', '4')
config.topktagger = getattr(cfg, 'topktagger', None)
config.topk = 50
# config.tagemb = getattr(cfg, 'tagemb', None) # cls/bert
config.tagemb = None
config.tagemb_gradient = getattr(cfg, 'tagemb_gradient', None)
config.category = getattr(cfg, 'category', 'vinvl')
config.tie_tag_weights = getattr(cfg, 'tie_tag_weights', False)


from src.layers.bert import ViTCAP
# TaggerEncDecCLSEmbSplitBertImgModel

# there might have logging warning error, ignore them
model = ViTCAP(config=config)

In [ ]:
%cd '/content/drive/MyDrive/ViTCAP'

!python run.py -c ./yaml/ViTCAP_Captioning_batch-size_512_encoder_vit_base_patch16_384_lr_1e-4_iter_60_vitbfocal20_bert_tokenizer_tags_ENC-DEC_multiplier_0.1.yaml


In [24]:
!kill 9


/bin/bash: line 1: kill: (9) - No such process


In [ ]:
/content/drive/MyDrive/ViTCAP/src/data_layer/dataset.py

In [ ]:
with open('/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/train.hw.tsv') as f:
    for line in f:
        key, str_hw = line.strip().split('\t')
        print(key, str_hw)

In [ ]:
# Load an image and apply data augmentation.

# TIMM style data pre-processing
class BGR2RGB(object):
    def __call__(self, im):
        return im[:, :, [2, 1, 0]]


def get_transform_vit_default(cfg):
    normalize = transforms.Normalize(
        mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

    trans = [
        BGR2RGB(),
        transforms.ToPILImage(),
    ]

    trans.extend([
        transforms.Resize(int(math.floor(cfg.test_crop_size / cfg.crop_pct)), PIL.Image.BICUBIC),
        transforms.CenterCrop(cfg.test_crop_size),
    ]),
    trans.extend([
        transforms.ToTensor(),
        normalize,
    ])
    transform = transforms.Compose(trans)

    return transform


transform = get_transform_vit_default(cfg)

In [ ]:
import csv
import json

def fix_json_in_tsv(input_file, output_file):
    with open(input_file, 'r') as tsv_file, open(output_file, 'w', newline='') as fixed_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        fieldnames = tsv_reader.fieldnames
        tsv_writer = csv.DictWriter(fixed_file, fieldnames=fieldnames, delimiter='\t')
        tsv_writer.writeheader()

        for row in tsv_reader:
            # Xác định cột chứa dữ liệu JSON không hợp lệ (ví dụ: 'json_data')
            json_column = 'json_data'
            if json_column in row:
                json_data = row[json_column]
                try:
                    # Cố gắng sửa lỗi cú pháp JSON
                    fixed_json = json.loads(json_data)
                    row[json_column] = json.dumps(fixed_json)
                except json.JSONDecodeError as e:
                    print(f"Lỗi cú pháp JSON trong dòng {row}: {e}")
            tsv_writer.writerow(row)

# Thực hiện sửa lỗi trong tệp TSV
input_file = '/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/train.hw.tsv'
output_file = '/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/fixed.tsv'
fix_json_in_tsv(input_file, output_file)



In [ ]:
/content/drive/MyDrive/ViTCAP/src/data_layer/transform.py

In [ ]:
import json

json_str = '{"key": "value", "nested": {"inner_key": "inner_value"}}'
try:
    json_obj = json.loads(json_str)
    # Kiểm tra cấu trúc JSON
    if isinstance(json_obj, dict):
        print("JSON object is a dictionary.")
    elif isinstance(json_obj, list):
        print("JSON object is a list.")
    # Nếu bạn muốn kiểm tra cụ thể cấu trúc, bạn có thể truy cập các phần tử bằng key hoặc index.
    print(json_obj["key"])
except json.JSONDecodeError as e:
    print("Invalid JSON:", e)

JSON object is a dictionary.
value


In [ ]:
import json

# Đường dẫn đến tệp JSON đầu vào và TSV đầu ra
json_file_path = '/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/output_json_file.json'
tsv_file_path = '/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/train.hw.tsv'

# Đọc dữ liệu từ tệp JSON
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Mở tệp TSV đầu ra để viết
with open(tsv_file_path, 'w') as tsv_file:
    # Viết tiêu đề cho tệp TSV

    # Viết dữ liệu từ JSON vào TSV với cấu trúc đúng
    for key, value in data.items():
        json_value = json.dumps(value)
        tsv_file.write(f"{key}\t{json_value}\n")

print(f"Dữ liệu đã được lưu vào tệp TSV: {tsv_file_path}")


Dữ liệu đã được lưu vào tệp TSV: /content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/train.hw.tsv


In [ ]:
import json

# Đường dẫn đến tệp TSV đầu vào và JSON đầu ra
tsv_file_path = '/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/train.hw.tsv'
json_file_path = '/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/output_json_file.json'

# Đọc dữ liệu từ tệp TSV
data = {}
with open(tsv_file_path, 'r') as tsv_file:
    for line in tsv_file:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            key, json_str = parts
            try:
                json_data = json.loads(json_str)
                data[key] = json_data
            except json.JSONDecodeError:
                print(f"Không thể phân tích dòng: {line}")

# Lưu dữ liệu dưới dạng JSON vào tệp mới
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"Dữ liệu đã được lưu vào tệp JSON: {json_file_path}")


Dữ liệu đã được lưu vào tệp JSON: /content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/output_json_file.json


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.9 MB/s eta 0:00:00


In [ ]:
 #wrap the model into a captioning pipeline
# seperate out the linear projection as image encoder, this makes the training/inference easier
def get_image_encoder_model(is_train=False):
    if cfg.image_encoder_type.startswith('VitEmb_'):
        # VitEmb_base32_384
        net = cfg.image_encoder_type[len('VitEmb_'):]

        if cfg.image_encoder_pretrained:
            logging.info('VIT image encoder loaded from pre-trained weight!  '
                         'Note that this might be replaced by pre-trained checkpoint later!')
        from src.pytorch_image_models import timm

        logging.info('Non-Patch Selection Mode.')
        model = timm.create_model(
            net,
            output_grid=True,
            pretrained=cfg.image_encoder_pretrained,
        )

        # clear out the following two modules
        model.norm = nn.Identity()
        model.blocks = nn.ModuleList()
        if not is_train:
            model.eval()

        from src.tools.torch_common import InputAsDict
        model = InputAsDict(model)

    else:
        raise NotImplementedError(self.cfg.image_encoder_type)
    return model

# image encoder: one linear projection
image_encoder = get_image_encoder_model(is_train=False)

# testing mode
tokenizer = get_tokenizer(cfg)
cls_token_id, sep_token_id, pad_token_id, mask_token_id, period_token_id = \
    tokenizer.convert_tokens_to_ids([
        tokenizer.cls_token,
        tokenizer.sep_token,
        tokenizer.pad_token,
        tokenizer.mask_token,
        '.',
    ])

test_extra_input = {
    'is_decode': True,
    'do_sample': False,
    'bos_token_id': cls_token_id,
    'pad_token_id': pad_token_id,
    'eos_token_ids': [sep_token_id],
    'mask_token_id': mask_token_id,
    # for adding od labels
    'add_od_labels': cfg.add_od_labels,
    'od_labels_start_posid': cfg.max_seq_a_length,
    # hyper-parameters of beam search
    'max_length': cfg.max_gen_length,
    'num_beams': cfg.num_beams,
    "temperature": cfg.temperature,
    "top_k": cfg.top_k,
    "top_p": cfg.top_p,
    "repetition_penalty": 1,
    "length_penalty": 1,
    "num_return_sequences": 1,
    "num_keep_best": 1,
}


model = ImageCaptioning(
    model,
    test_extra_input,
    tokenizer=tokenizer,
    bert_tokenizer=tokenizer,
    image_encoder=image_encoder,
    cfg=cfg,
)

helpers.py  <All keys matched successfully>


In [ ]:
# Load an image and apply data augmentation.

# TIMM style data pre-processing
class BGR2RGB(object):
    def __call__(self, im):
        return im[:, :, [2, 1, 0]]


def get_transform_vit_default(cfg):
    normalize = transforms.Normalize(
        mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

    trans = [
        BGR2RGB(),
        transforms.ToPILImage(),
    ]

    trans.extend([
        transforms.Resize(int(math.floor(cfg.test_crop_size / cfg.crop_pct)), PIL.Image.BICUBIC),
        transforms.CenterCrop(cfg.test_crop_size),
    ]),
    trans.extend([
        transforms.ToTensor(),
        normalize,
    ])
    transform = transforms.Compose(trans)

    return transform


transform = get_transform_vit_default(cfg)

In [ ]:
# load from the checkpoint
from src.tools.opt.checkpoint import align_and_update_state_dicts
from src.tools.qd_pytorch import load_model_state_ignore_mismatch
checkpoint = torch.load("/content/drive/MyDrive/ViTCAP/data/ViTCAPCOCO/checkpoint/Logit_Vilt_captioning_testing_batch-size_512_encoder_vit_base_patch16_384_lr_1e-4_iter_60_vitbfocal20_bert_tokenizer_tags_ENC-DEC_multiplier_0.1_expand_tag-classifier_emb.zip")


def strip_prefix_if_present(state_dict, prefix):
    from src.tools.torch_common import remove_prefix
    return remove_prefix(state_dict, prefix)

model_state_dict = model.state_dict()

# if the state_dict comes from a model that was wrapped in a
# DataParallel or DistributedDataParallel during serialization,
# remove the "module" prefix before performing the matching
loaded_state_dict = strip_prefix_if_present(checkpoint['model'], prefix="module.")
align_and_update_state_dicts(model_state_dict, loaded_state_dict)

# use strict loading
# model.load_state_dict(model_state_dict)

# use non-strict loading
load_model_state_ignore_mismatch(model, model_state_dict)

model.cuda()
model.eval()

FileNotFoundError: ignored

In [ ]:
import torch
print(torch.__version__)


2.0.1+cu118


In [ ]:
import pandas as pd
import json
import numpy as np
import regex as re


%cd '/content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark/tools/mini_tsv/Images'

/content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark/tools/mini_tsv/Images


In [ ]:
%cd '/content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark/tools/mini_tsv'
df = pd.read_csv('captions.txt', sep=',')
print(df.shape[0])
df.head()

/content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark/tools/mini_tsv
40455


,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [ ]:
split = 'test'

# Remove file extension '.jpg'
df['image'] = df['image'].apply(lambda row : row[:-4])

# Make clean captions
#reg = re.compile('[^a-zA-Z, ]+')
#df['caption'] = df['caption'].apply(lambda row : re.sub(reg, '', row).strip()+'.')
df['caption'] = df['caption'].apply(lambda row : row.strip())

In [ ]:
df['image'].head()

0    1000268201_693b08cb0e
1    1000268201_693b08cb0e
2    1000268201_693b08cb0e
3    1000268201_693b08cb0e
4    1000268201_693b08cb0e
Name: image, dtype: object

In [ ]:
df['caption'].head()


0    A child in a pink dress is climbing up a set o...
1                A girl going into a wooden building .
2     A little girl climbing into a wooden playhouse .
3    A little girl climbing the stairs to her playh...
4    A little girl in a pink dress going into a woo...
Name: caption, dtype: object

In [ ]:
# Create image annotations
images = [{"file_name" : df['image'].iloc[i],"id" : df['image'].iloc[i]} for i in range(df.shape[0])]
annotations ={"images" : images}
# Create coco_format
coco_format = {"info": "dummy",
               "type": "captions",
               "licenses": "dummy",
               "images" : images}

In [ ]:
# dump json
with open(split+"_caption.json", "w") as f:
  json.dump(annotations, f, separators=(', ', ': '), ensure_ascii = False)

with open(split+"_caption_coco_format.json", "w") as f:
  json.dump(coco_format, f, separators=(', ', ': '), ensure_ascii = False)

In [ ]:
%cd /content/drive/MyDrive/ViTCAP/data


/content/drive/MyDrive/ViTCAP/data


oscar demo

In [ ]:
# install pytorch 1.7
! pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install apex
%cd /content/drive/MyDrive/ViTCAP/data/apex
! python setup.py install --cuda_ext --cpp_ext

# install oscar
%cd /content/drive/MyDrive/ViTCAP/data/Oscar
! pip install pycocoevalcap
! chmod u+r+x get_stanford_models.sh
! ./get_stanford_models.sh
%cd ..
! python setup.py build develop
%cd /content/drive/MyDrive/ViTCAP/data/Oscar
# install requirements
! pip install -r requirements.txt

In [ ]:
%cd /content/drive/MyDrive/ViTCAP/data/Oscar
# install requirements
! pip install -r requirements.txt

In [ ]:
%cd /content/drive/MyDrive/ViTCAP/data/Oscar

/content/drive/MyDrive/ViTCAP/data/Oscar


sgg

In [ ]:
%cd '/content/drive/MyDrive/ViTCAP/data'
import sys
import os
import os.path as op
import yaml
import pandas as pd
import ast
import json
import base64
import torch
import numpy as np
np.set_printoptions(suppress=True, precision=4)

In [ ]:
#%cd '/content/drive/MyDrive/ViTCAP/data'

# Clone repository và tải các tập tin cần thiết
#!git clone https://github.com/microsoft/scene_graph_benchmark.git

#!wget https://drive.google.com/file/d/1nvu8y4zZFbJqSqLdQClvMyzsbBeX-oaQ/view

#!wget https://drive.google.com/file/d/1M1nPtMPHS1GXx5HvKMMDxOksxQgwX14_/view

%cd /content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark

  # Cài đặt các phụ thuộc
!pip install ninja yacs>=0.1.8 cython matplotlib tqdm opencv-python numpy>=1.19.5

!pip install -U PyYAML

  # Cài đặt pycocotools
!pip install pycocotools

  # Cài đặt cityscapesScripts
!python -m pip install cityscapesscripts

  # Cài đặt thư viện với symbolic links
!python setup.py build develop

In [ ]:
!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.6 -c pytorch -c nvidia



In [ ]:
%cd '/content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark'

import os.path
import yaml
! python /content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark/tools/mini_tsv/tsv_demo.py

TSV_DIR = '/content/drive/MyDrive/ViTCAP/data/scene_graph_benchmark/tools/mini_tsv/data/'
# Create .tsv files for our images

# Create .yaml file for connecting .tsv files
yaml_dict = {"img" : "train.tsv",
            "label" :  "train.label.tsv",
            "hw" : "train.hw.tsv",
            "linelist" : "train.linelist.tsv"}
with open(os.path.join(TSV_DIR, 'train.yaml'), 'w') as file:
        yaml.dump(yaml_dict, file)